In [3]:
import torch    
torch.manual_seed(123)
import random
random.seed(123)
# import torch.multiprocessing
# torch.multiprocessing.set_start_method("spawn")

import torch.nn as nn
import os
import shutil
import itertools

from chofer_torchex.utils.data.collate import dict_sample_target_iter_concat
from chofer_torchex.utils.functional import collection_cascade, cuda_cascade
from chofer_tda_datasets import AnonEigenvaluePredict
from chofer_tda_datasets.transforms import Hdf5GroupToDict
from jmlr_2018_code.utils import *
from chofer_torchex.nn.slayer import SLayerRationalHat, LinearRationalStretchedBirthLifeTimeCoordinateTransform, prepare_batch
from sklearn.model_selection import ShuffleSplit
from collections import Counter, defaultdict

%matplotlib notebook

from torch.utils.data import DataLoader

os.environ['CUDA_VISIBLE_DEVICES'] = str(3)


class AnonCollate:   
    def __init__(self, dataset, cuda=True):
        self.cuda = cuda
        self.dataset = dataset
        
    def __call__(self, sample_target_iter):
        x, y = dict_sample_target_iter_concat(sample_target_iter)

        for k in self.dataset.keys_not_essential:
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 2)
            
        for k in self.dataset.keys_essential:
            batch_view = x[k]
            x[k] = prepare_batch(batch_view, 1)   
            
        y = torch.Tensor(y)

        if self.cuda:
            # Shifting the necessary parts of the prepared batch to the cuda
            x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                 for k, v in x.items()}

            y = y.cuda()

        return x, y
    

class train_env:
    n_epochs = 100
    lr_initial = 0.01
    lr_epoch_step = 20
    batch_size = 5
    train_size = 0.9
    nu = 0.01
    n_target_bins = 100

    
dataset = AnonEigenvaluePredict(data_root_folder_path='/scratch1/chofer/jmlr2018_data/')
dataset.keys_essential = ('dim_0_ess', 'dim_1_ess')
dataset.keys_not_essential = ('dim_0',)

def coordinate_transform(x):
    t = LinearRationalStretchedBirthLifeTimeCoordinateTransform(nu=train_env.nu)    
    for k in dataset.keys_not_essential:
        x[k] = t(x[k])   
        
    return x
    
def normalize(x):
    dim_1 =  x['dim_0']/x['dim_0'][:, 1].max() 
    dim_1 = dim_1[(dim_1[:,1] - dim_1[:,0] > train_env.nu).nonzero().squeeze()]
    
    x['dim_0'] = dim_1    
    
    x['dim_0_ess'] = (x['dim_0_ess']/x['dim_0_ess'].max()).unsqueeze(1)
    x['dim_1_ess'] = (x['dim_1_ess']/x['dim_1_ess'].max()).unsqueeze(1)
    return x

        
dataset.data_transforms = [Hdf5GroupToDict(),
                           numpy_to_torch_cascade,
                           coordinate_transform,
                           normalize
                           ]
def histogramize(x):
    return np.histogram(x, normed=True, bins=train_env.n_target_bins,range=(0,2))[0].tolist()
    

dataset.target_transforms = [histogramize]                      




In [4]:
lengths = defaultdict(list)
for x, y in dataset:
    for k, v in x.items():
        lengths[k].append(v.size(0))
    

KeyboardInterrupt: 

In [5]:
lengths

defaultdict(list,
            {'dim_0': [1610,
              2577,
              2419,
              2526,
              2661,
              1933,
              1786,
              3380,
              1885,
              1594,
              1631,
              1706,
              1944,
              1362,
              2392,
              1519,
              1723,
              2014,
              1840,
              1713,
              1777,
              1842,
              1845,
              2051,
              1426,
              1130,
              1700,
              1502,
              1813,
              1732,
              2327,
              1681,
              1556,
              2741,
              1720,
              2037,
              2522,
              2585,
              1901,
              2091,
              1875,
              1882,
              1761,
              1345,
              2078,
              2002,
              1299,
              1660,
             

In [ ]:
reddit_collate = AnonCollate(dataset, cuda=False)         

In [ ]:
def LinearCell(n_in, n_out):
    m = nn.Sequential(nn.Linear(n_in, n_out), 
                      nn.BatchNorm1d(n_out), 
                      nn.ReLU())
    m.out_features = m[0].out_features
    return m


def Slayer(n_elements, point_dim):
    return SLayerRationalHat(n_elements, point_dimension=2, radius_init=0.25)   


class AnonModel(nn.Module):    
    def __init__(self):
        super().__init__()  
        
        dim_0_n_elements = 50
        dim_0_ess_n_elements = 10
        dim_1_ess_n_elements = 10
        
        self.dim_0 = Slayer(dim_0_n_elements, 2)
        self.dim_0_ess = Slayer(dim_0_ess_n_elements, 1)
        self.dim_1_ess = Slayer(dim_1_ess_n_elements, 1)      
        
        self.dim_0_linear = LinearCell(dim_0_n_elements, dim_0_n_elements)
        self.dim_0_ess_linear = LinearCell(dim_0_ess_n_elements, dim_0_ess_n_elements)
        self.dim_1_ess_linear = LinearCell(dim_1_ess_n_elements, dim_1_ess_n_elements)
        
        cls_in_size = self.dim_0_linear.out_features + self.dim_0_ess_linear.out_features + self.dim_1_ess_linear.out_features
        self.classifer = nn.Sequential(
                                         nn.BatchNorm1d(cls_in_size),
#                                        LinearCell(cls_in_size, cls_in_size),
#                                        nn.Dropout(0.2),
#                                        LinearCell(cls_in_size, int(cls_in_size/2)),                                        
                                         nn.Linear(cls_in_size, train_env.n_target_bins))
                         
    def forward(self, x):
        x_dim_0 = self.dim_0(x['dim_0'])        
        x_dim_0 = self.dim_0_linear(x_dim_0)
        
        x_dim_0_ess = self.dim_0_ess(x['dim_0_ess'])   
        x_dim_0_ess = self.dim_0_ess_linear(x_dim_0_ess)
        
        x_dim_1_ess = self.dim_1_ess(x['dim_1_ess'])       
        x_dim_1_ess = self.dim_1_ess_linear(x_dim_1_ess)
        
        x = torch.cat([x_dim_0, x_dim_0_ess, x_dim_1_ess], dim=1)
        
        x = self.classifer(x)        
        
        return x
        
    def centers_init(self):
        dim_0 = []
        for i in range(self.dim_0.centers.size(0)):
            x = random.uniform(0, 1)
            y = random.uniform(0, 1-x)
            dim_0.append((x,y))
        self.dim_0.centers.data = torch.Tensor(dim_0)
        
        self.dim_0_ess.centers.data.uniform_(0, 1)
        self.dim_1_ess.centers.data.uniform_(0, 1)


In [ ]:
stats_of_runs = []
def experiment():      
    splitter = ShuffleSplit(n_splits=10, 
                            train_size=train_env.train_size, 
                            test_size=1-train_env.train_size, 
                            random_state=123)
    
    train_test_splits = list(splitter.split(X=dataset.targets, y=dataset.targets))
    train_test_splits = [(train_i.tolist(), test_i.tolist()) for train_i, test_i in train_test_splits]
    
    for run_i, (train_i, test_i) in enumerate(train_test_splits):
        print('')
        print('Run', run_i)
        
        model = AnonModel()
        model.centers_init()
        model.cuda()

        stats = defaultdict(list)
        stats_of_runs.append(stats)
        
        opt=torch.optim.SGD(model.parameters(), lr=train_env.lr_initial, momentum=0.9)

        for i_epoch in range(1, train_env.n_epochs+1):      

            model.train()

            train_sampler = [i for i in train_i] 
            random.shuffle(train_sampler)
            
            dl_train = DataLoader(dataset,
                              batch_size=train_env.batch_size, 
                              collate_fn=reddit_collate,
                              sampler=train_sampler, 
                              num_workers=10)

            dl_test = DataLoader(dataset,
                                 batch_size=train_env.batch_size, 
                                 collate_fn=reddit_collate, 
                                 sampler=test_i, 
                                 num_workers=10)

            epoch_loss = 0

            if i_epoch % train_env.lr_epoch_step == 0:
                adapt_lr(opt, lambda lr: lr*0.5)

            for i_batch, (x, y) in enumerate(dl_train, 1):  
                
                x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                     for k, v in x.items()}

                y = y.cuda()
                
                y = torch.autograd.Variable(y)

                def closure():
                    opt.zero_grad()
                    y_hat = model(x)            
                    loss = nn.functional.mse_loss(y_hat, y)   
                    loss.backward()
                    return loss

                loss = opt.step(closure)

                epoch_loss += float(loss)
                stats['loss_by_batch'].append(float(loss))
                stats['centers'].append(model.dim_0.centers.data.cpu().numpy())

                print("Epoch {}/{}, Batch {}/{}".format(i_epoch, train_env.n_epochs, i_batch, len(dl_train)), end="       \r")

            stats['train_loss_by_epoch'].append(epoch_loss/len(dl_train))

            print("\n\r testing...")
            model.eval()    
            true_samples = 0
            seen_samples = 0
            epoch_test_loss = 0
            for i_batch, (x, y) in enumerate(dl_test):
                
                x = {k: collection_cascade(v,
                                       lambda x: isinstance(x, tuple),
                                       lambda x: (x[0].cuda(), x[1].cuda(), x[2], x[3]))
                     for k, v in x.items()}

                y = y.cuda()

                y_hat = model(x)
                epoch_test_loss += float(nn.functional.mse_loss(y_hat, torch.autograd.Variable(y.cuda())).data) 
                seen_samples += y.size(0)
                
            avg_epoch_test_loss = epoch_test_loss/len(dl_test)   
            stats['test_loss_by_epoch'].append(avg_epoch_test_loss)            
            print(avg_epoch_test_loss)
        
        stats['tain_i'] = train_i
        stats['test_i'] = test_i
        stats['model'] = model.cpu()
        
experiment()        

In [ ]:
plt.figure()
stats = stats_of_runs[0]
c_start = stats['centers'][0]
c_end = stats['centers'][-1]

plt.plot(c_start[:,0], c_start[:, 1], 'bo', label='center initialization')
plt.plot(c_end[:,0], c_end[:, 1], 'ro', label='center learned')

all_centers = numpy.stack(stats['centers'], axis=0)
for i in range(all_centers.shape[1]):
    points = all_centers[:,i, :]
    plt.plot(points[:, 0], points[:, 1], '-k')
    
plt.legend()

# for c_i_start, c_i_end in zip(c_start, c_end):
#     points = numpy.stack([c_i_start, c_i_end], axis=0)
#     plt.plot(points[:,0], points[:,1], '-k')
    




plt.figure()
plt.plot(stats['train_loss_by_epoch'], label='train_loss')
plt.plot(stats['test_loss_by_epoch'], label='test_loss')


plt.legend()
plt.show()